In [8]:
import pandas as pd;
import math;
import numpy;

            

Coordenadas Montes Claros - MG
Latitude: 16° 43' 41" / -16.6855037 Sul
Longitude: 43° 51' 54" / -43.8276535 Oeste
Altitude: 638 Metros

In [12]:
datum_parametros = pd.read_csv("datum_parametros.csv")
display(datum_parametros)

,id,datum,semi_eixo,achat,deltax,deltay,deltaz
0,0,NaN,0.0,0.000000,0.00,0.00,0.00
1,1,SAD69,6378160.0,0.003353,-67.35,3.88,-38.22
2,2,CorregoAlegre,6378388.0,0.003367,-206.05,168.28,-3.82
3,3,AstroChua,6378388.0,0.003367,-144.35,243.37,-33.22
4,4,WGS84,6378137.0,0.003353,0.00,0.00,0.00
5,5,SIRGAS2000,6378137.0,0.003353,0.00,0.00,0.00


In [ ]:
# Função que converte coordenadas geográficas para UTM
# return x = Longitude UTM
# return y = Latitude UTM
# zona UTM 
def geo_to_utm(lon, lat, semi_eixo, achat, hemis, lon_mc):
    pi = pi();
    
    if hemis == "norte":
        offy = 0.;
    else:
        offy = 10000000.;
        
    #convert lat long em UTM
    k0 = 1. - (1./2500.);
    equad = 2.*achat - pow(achat,(float)(2));
    elinquad = equad/(1. - equad);
    
    aux1 = equad*equad;
    aux2 = aux1*equad;
    aux3 = math.sin(((float)(2))*lat);
    aux4 = math.sin(((float)(4))*lat);
    aux5 = math.sin(((float)(6))*lat);

In [9]:
# Função que define o meridiando Central da Zona UTM com base na Longitude informada (necessita revisão)
# @param lon = longitude em coordenadas geográficas (graus decimais).
def define_mer_cent(lon):
    
        if lon != 0.:
            sinal = (int)(lon/abs(lon))
        else:
            mc1_utm = 3.;
            mc2_utm = -3.;
        
        ind1=0
        k=0
        ind2=6
        
        while((abs(lon))>((float)(ind2))):
            mc = ind1 + 3.;
            k+=1;
            ind1 = 6 * k;
            ind2 = ind1 + 6;
        
        if ((abs(lon)) < ((float)(mc))) and (lon != 0.):
            mc1_utm = mc2_utm = ((float)(mc));
            print("Entrou no 1° if!!!!");
        
        if ((abs(lon)) > ((float)(mc))) and ((abs(lon)) != ((float)(mc) + 3.)):
            mc1_utm = mc2_utm = ((float)(mc));
            print("Entrou no 2° if!!!!");
        
        if ((abs(lon)) == ((float)(mc) + 3.)) and ((abs(lon)) != 180.):
            mc1_utm = ((float)(mc));
            mc2_utm = ((float)(mc)) + 6.;
            print("Entrou no 3° if!!!!");
        
        if (lon == 180.):
            mc1_utm = 177.;
            mc2_utm = -177.;
            print("Entrou no 4° if!!!!");
        
        if (lon == -108.):
            mc1_utm = -177.;
            mc2_utm = 177.;
            print("Entrou no 5° if!!!!");
        
        if ((abs(lon)) == ((float)(mc))):
            mc1_utm = mc2_utm = ((float)(mc));
            print("Entrou no 6° if!!!!");
        
        if (lon != 0.) and ((abs(lon)) != 180):
            mc1_utm *= sinal;
            mc2_utm *= sinal;
            print("Entrou no 7° if!!!!");
            
        print("MC: ", mc)
        print("MC1_UTM: ", mc2_utm)
        print("MC2_UTM: ", mc2_utm)
            
            
        

In [10]:
#main code

define_mer_cent(-16.6855037)

Entrou no 2° if!!!!
Entrou no 7° if!!!!
MC:  9.0
MC1_UTM:  -9.0
MC2_UTM:  -9.0
